![](https://media2.giphy.com/media/ZnaODGlDcAISY/200.webp?cid=790b7611bcaa870871a583a5bfed96538abe847d7a7a9656&rid=200.webp)

# Dr David G. Fearn - David Fearn, was internationally recognised as the father of ion propulsion in spacecraft, which he developed at the Royal Aircraft Establishment, Farnborough (now QinetiQ); his work enables telecommunications satellites to be positioned more accurately and, by saving on heavy rocket fuel, has made feasible missions to deep space that were previously impossible. Self-effacing but generous in his praise for others, Fearn tirelessly supported research at British universities; many of the PhD students he taught from Oxford and Southampton, and at Surrey University, where he held a visiting Chair, now hold influential positions in industry, academe and the ESA. He published more than 250 technical papers, many of them ground-breaking.https://www.imperial.ac.uk/news/25974/dr-david-fearn-1960-phd-1963/

Until the development of ion propulsion, Earth satellites and deep space probes could be manoeuvred only by thruster rockets, using large amounts of fuel.
Dual Stage Four Grid (DS4G) Ion Engine for Very High Velocity Change Missions IEPC 2009-157 31st International Electric Propulsion Conference, Ann Arbor, Michigan, USA  September 20-24, 2009 
 
R. Intini Marques1, S.B. Gabriel2 - University of Southampton, Astronautics Research Group, Southampton, SO17 1BJ, United Kingdom

The Dual-Stage 4-Grid (DS4G) is an electrostatic ion thruster design developed by the European Space Agency, in collaboration with the Australian National University. The design was derived by D. Fearn from Controlled Thermonuclear Reactor experiments that use a 4-grid mechanism to accelerate ion beams. https://en.wikipedia.org/wiki/Dual-Stage_4-Grid
Very high delta-V missions to the edge of the solar system and beyond enabled by the dual-stage 4-grid ion thruster concept
Cristina Bramanti, Dario Izzo, T. Samaraee, Rosanne Walker, David G. Fearn - Physics2009

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

#Map Codes Paul Mooney https://www.kaggle.com/paultimothymooney/how-to-get-started-with-the-earth-engine-data

#and Ragnar https://www.kaggle.com/ragnar123/exploratory-data-analysis-and-factor-model-idea

In [ ]:
import rasterio as rio
import folium
import tifffile as tiff

In [ ]:
def plot_points_on_map(dataframe,begin_index,end_index,latitude_column,latitude_value,longitude_column,longitude_value,zoom):
    df = dataframe[begin_index:end_index]
    location = [latitude_value,longitude_value]
    plot = folium.Map(location=location,zoom_start=zoom)
    for i in range(0,len(df)):
        popup = folium.Popup(str(df.primary_fuel[i:i+1]))
        folium.Marker([df[latitude_column].iloc[i],df[longitude_column].iloc[i]],popup=popup).add_to(plot)
    return(plot)

In [ ]:
def overlay_image_on_puerto_rico(file_name,band_layer,lat,lon,zoom):
    band = rio.open(file_name).read(band_layer)
    m = folium.Map([lat, lon], zoom_start=zoom)
    folium.raster_layers.ImageOverlay(
        image=band,
        bounds = [[18.6,-67.3,],[17.9,-65.2]],
        colormap=lambda x: (1, 0, 0, x),
    ).add_to(m)
    return m

In [ ]:
def plot_scaled(file_name):
    vmin, vmax = np.nanpercentile(file_name, (5,95))  # 5-95% stretch
    img_plt = plt.imshow(file_name, cmap='gray', vmin=vmin, vmax=vmax)
    plt.show()

In [ ]:
def split_column_into_new_columns(dataframe,column_to_split,new_column_one,begin_column_one,end_column_one):
    for i in range(0, len(dataframe)):
        dataframe.loc[i, new_column_one] = dataframe.loc[i, column_to_split][begin_column_one:end_column_one]
    return dataframe

#How to read tif files 

In [ ]:
from skimage.io import imread
image = imread('/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/s5p_no2_20190107T163828_20190113T182723.tif')
print (image.shape)
plt.imshow(image[:,:,0], cmap = 'winter')
plt.axes = False

In [ ]:
from skimage.io import imread
image = imread('/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/s5p_no2_20190108T161929_20190114T180349.tif')
print (image.shape)
plt.imshow(image[:,:,0], cmap = 'ocean')
plt.axes = False

In [ ]:
from skimage.io import imread
image = imread('/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/s5p_no2_20190116T171037_20190123T104808.tif')
print (image.shape)
plt.imshow(image[:,:,0], cmap = 'terrain')
plt.axes = False

#All codes to read .tif I copied from JyeSR https://www.kaggle.com/jyesawtellrickson/data-measurement-levels

In [ ]:
data_path = '/kaggle/input/ds4g-environmental-insights-explorer'
from datetime import datetime, timedelta
os.listdir(data_path + '/eie_data')

In [ ]:
gldas_files = os.listdir(data_path + '/eie_data/gldas')
gldas_dates = [datetime.strptime(g, 'gldas_%Y%m%d_%H%M.tif') for g in gldas_files]

gfs_files = os.listdir(data_path + '/eie_data/gfs')
gfs_dates = [datetime.strptime(g, 'gfs_%Y%m%d%H.tif') for g in gfs_files]

s5p_files = os.listdir(data_path + '/eie_data/s5p_no2')
s5p_dates = [datetime.strptime(g[:16], 's5p_no2_%Y%m%d') for g in s5p_files]

In [ ]:
all_dates = pd.DataFrame(columns=['dataset', 'datetime']).append(
    pd.DataFrame(gldas_dates, columns=['datetime']) \
        .assign(dataset = 'gldas'), sort=True
).append(
    pd.DataFrame(gfs_dates, columns=['datetime']) \
        .assign(dataset = 'gfs'), sort=True
).append(
    pd.DataFrame(s5p_dates, columns=['datetime']) \
        .assign(dataset = 's5p'), sort=True
).append(
    pd.DataFrame([datetime(y, 1, 1) for y in range(2013, 2018)], columns=['datetime']) \
    .assign(dataset = 'gppd'), sort=True
).assign(date = lambda x: x.datetime.apply(lambda x: x.date()))

In [ ]:
all_dates

In [ ]:
all_dates.groupby('dataset').date.agg(
    min=min,
    max=max,
    measurement_period=  lambda x: (x.max()-x.min()).days+1,
    measurement_count= 'count',
    measurements_per_day= lambda x: x.count() / ((x.max()-x.min()).days+1)
).transpose()

In [ ]:
# Get the date index to work with
daily_data = pd.date_range('2018-07-01', '2019-06-30').to_frame() \
    .merge(all_dates.groupby(['dataset', 'date']).date.count().unstack(level=0),
        left_index=True,
        right_index=True) \
    .drop(columns=[0, 'gppd'], axis=1)

sns.heatmap(daily_data.transpose())
fig = plt.gcf()
fig.set_size_inches(11,3)

In [ ]:
dgfearn = pd.read_csv('/kaggle/input/ds4g-environmental-insights-explorer/eie_data/gppd/gppd_120_pr.csv')

In [ ]:
dgfearn.head()

In [ ]:
dgfearn.dtypes

In [ ]:
dgfearn = split_column_into_new_columns(dgfearn,'.geo','latitude',50,66)
dgfearn = split_column_into_new_columns(dgfearn,'.geo','longitude',31,48)
dgfearn['latitude'] = dgfearn['latitude'].astype(float)
a = np.array(dgfearn['latitude'].values.tolist()) # 18 insted of 8
dgfearn['latitude'] = np.where(a < 10, a + 10, a).tolist()
lat = 18.200178; lon = -66.664513
plot_points_on_map(dgfearn, 0, 425, 'latitude', lat, 'longitude', lon, 9)

In [ ]:
image = '/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/s5p_no2_20180701T161259_20180707T175356.tif'
latitude=18.1429005246921; longitude=-65.4440010699994
overlay_image_on_puerto_rico(image,band_layer=7,lat=latitude,lon=longitude,zoom=8)

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'https://techcrunch.com/wp-content/uploads/2014/09/screen-shot-2014-09-26-at-9-57-06-am.png?w=1390&crop=1',width=400,height=400)

Inspired by The Folium Map above I joined the episode of South Park https://techcrunch.com/2014/09/26/south-park-riffs-on-startups-kickstarter-and-brilliantly-tackles-washington-redskins-name-controversy/

"This South Park episode is a hilarious smack down on the NFL’s inability to do anything, the Washington Redskins name controversy and startup culture. The episode kicks off with the gang listing off possible names for their startup like “Dense Boner Forest” and “Furry Balls Plopped Menacingly On The Table, Inc.” You will possibly be offended and possibly laugh your ass off. This is South Park, after all."

"Some of the highlights include Stan explaining to his dad that he doesn’t need school anymore because he’s doing a startup now. “We don’t want to do anything,” says Stan. Then Kyle chimes in and explains, “That’s why we want to have a startup company.” Cartman gets inspired to call their do-nothing startup the “Washington Redskins.” 

"The gang creates a super successful Kickstarter campaign to fund their startup and Cartman goes into full Tim Cook mode as CEO. “F*** you. Those words mean a great deal to us. They help us express as a company how we see things differently,” Cartman tells a conference: “And now our company is thrilled to show you all the latest innovations we’ve come up with. To begin with, we have moved the couch from the left side of the office, to the right side,” he tells them.

"True to South Park style, this episode calls attention to many of the major issues dominating the news and pop culture in the way it does best." You can enjoy the episode: “Go Fund Yourself” https://techcrunch.com/2014/09/26/south-park-riffs-on-startups-kickstarter-and-brilliantly-tackles-washington-redskins-name-controversy/  by Sarah Buhr

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxAQEBUSDxAVFQ8WFRYVFhAWFRAVEBUVFRUXFhUVFRUYHSggGB4lHRYVITEhJSkrLi4uFx8zODMtNygtLisBCgoKDg0OGhAQGy0gHR4tLS0tLSsvLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tKy0tLS0tLS0tLS0tLf/AABEIAKgBLAMBEQACEQEDEQH/xAAbAAABBQEBAAAAAAAAAAAAAAAAAQIDBAUGB//EAEQQAAIBAgMEBgULAgUDBQAAAAECAwARBBIhBTFBUQYTMmFxkRQigaHRFSMzQlKCkrHBwvBTohZEYnKTQ4PhByRUstL/xAAaAQEAAwEBAQAAAAAAAAAAAAAAAQIEAwUG/8QANxEAAgECBAMFBgYCAgMAAAAAAAECAxEEEiExBUFRE2FxgZEiMqGxwdEUM0JS4fAGFYLxIzRT/9oADAMBAAIRAxEAPwDXrWZQtQC0AXoAoBLUBjD6X7/7qqWK+I7BrzTYaoqCRaEkGL3Dx/Q0IKtAFSQFCQoGFQAqSBaEiUAUAVACpAUIChIUIFoBKAKEhQBUAKEBUkhQBQgKEhQBQgSgMzC7enTQsHHJhc+Y1863KTOLppmtheksZ0kUoeY9Zfj7quplHSfI1cPi45Po3Vu4HXy31ZNM5tNbk9SQFAFAYo+l+/8AuqpYr4jsGvNNhqCoLC0BBi9w8f0NCGVakgKAWhKEqALQBQCVJAVBItAFSBKEBUEhQC0IEoSFAFSBagCGhBG0tqzzryj+nQyVMTKOrg7CLLeqfilfbQoscr7aDpL5Tl38L7vA1ri09eRszXjeJFg8Wsg00Yb15VecMrIhUUiPFY0KcqkZuJO5RXJytoZcRjFCWSO/NvZf3oQNtQDQAseegHsAqM5nfE0tIpyfXYUYqc7otO8EfmaXl0LrEYuWqpkq4ibjD/cBVrvodFWxNtafxMSt5rChIoPEb+dCC/hdszx7nzDk3re/f76spMq4JmthukynSWMjvXUeR1qymUdLoa2F2hDJ2JATy3N5HWrJo5uLRnj6X7/7qgFfEdg/zjXmmw1BUEhQkhxe4eP6UIZVqQFCBaEiUAVAFqSAqAJQkKAKAKEC1JIlAFAFCAoSBNqgXKE21UX6khHMKQPfalzLPFRjyb8is+1437MrRnkVUqfHeffUXRxeLhLaTj5DIdsgMVlI/wB6aqfZXCVGM5XkzLnjKo+1d+9bFwT5iCq5kI7XZI89/srlVo0ofqs+m4qwpL3GTdbbvvbdyJtf41SjU0cHzT9S1GvljKD2aZmo3VTnlr5HX+eFepSqdpQUny+h1o1be2/Mqu9ySd5N6yXueLObqScnzG68qFSSPEOu4n2En3UuzrCtUh7smi7HtY29ZbnmLj9KupnoQ4nJK0o3fjY6OfY2Hf8A6YB5rdfy0r18qNym0Z8/RlfqSEdzAH3i1RkLqr1M+fo/Ou4Bh3HXyNqrkZZVEUJsM6dtGXxBAqti6aZCKEhQFnD4yRCCrbiDY6j30uyrimbeI7J/nGsR0NQVBItAQ4vcPH9DQFWgYUCEoAqQLQBQgShIVACpAtAJUA67odsqKSJ3ljVrvlXML2CgXt7T7q6wimtSsma5wmzxEZcsHUi95fU6sWOU3bdv08avlRW7I1+TDEZgcL1CmxmzRdUDyL3sKnKhdksMWz3VGQYdkkJEbDqishALEIR2rBWOnAGmVEXA/J4jWQ+jCJlLrJ8zkZVGYsrbiANb0yom4YCTZ85K4c4aRlAJEfUuQDuJC7qWRFxsOM2a4cpJhWWMFpCGgIRRvLkdkd5pZC5ZxGCwcSF5IoEjUXZ2SJVUDiWIsBSyK5I9B3UYUMqZIQ7AlVyxZmC2zFRbUC43cxSyGWPQlw6Qut4xGy3IuoQrdSVYXHIgg94NMq6E5V0PNuk8yti5CoACuE0FhoAvwryMb+bbuPMxTXaWXI5baTfOHuA+P61ooPLh7dX/AH5GapO1Jx5v+/QracTrVTFvsTwxE+stnA1K8R7OI8Kvle5pp4eTWeKzW3X8Es2EuvWRarxXiOY76OPNHaphFOHaUdunNEaRxEXLMp5AXHjUaHGH4eSvJuL8LndmvbPXCgCgFoCrPsyB+1Et+YFj5ioyospNGbN0aiPYdl8mFVyF1VZjSbJkDZVIbW3LjbjVXFl1URqYnsn+cawHY1BUFhaEEGL3Dx/SgKtCAqSQqAFCAqSQqCAoSFAgoAoAqSDsZ9n4v5PiTChD6pkliYuryhgXESsvZzEgHdppcXvXeKsij3IJej2J+TMDgQoBWTDid1syIkN5i1m7V3RBbm3KrEEeI6PvhcVFOIZcZHmnllCjDh/SXWNIpOqJRLKiMgtuzE8b0BmS9Hccod0gtI0e0MQsaMhSPE4zJDDGpuBmWLOxI9W5bXWgNHo50ZnwuMZGjz4DDQzei3sSxxTRPJHY/ZaJxrwkFAXtk7MxGF2OVSL/AN+cPK5QBcxxEueTKTfg7238KAxtp9C5IsIijPiJRHh8KURIUy4YTRPPYXGdmEdrsfK5oDS6Q4TF7TyRLhuqwoVpJUxNl61zmSJLRFtFI6w349X30BjvsbaLrh8V1LDGYHB4bqUYj52a7jFxmx+tGqr4sKA6forgmwuGjSVZVeHDRByzfMM5DSylBfVg5YFiOVAebriC4eRiCDIzA8bZzv8AKvHxNpVdNzx6klKbZlzRkDO9hm1tx57q7ycc+SPLQy14tz08EVGxA5GtNKWTdHelGVJe1DzJ8Lj8u5iut7ZQQfE10lUi90aqeLiudvI2cDjIn7JAYm5Xcb91646cjbQq0nfJu9RuI2cGa6m1+FuNVcLmWvw2NSblF2udYK9c6C0AUAWoAtQBQGKPpfv/ALqqWK+I7B/nGvNNpqCoJFoCDF7h4/oaEFahAUJEqQLUAKAKAKEBQkKEBQEmGhMjqg3syr+IgfrUoM9RkwcTWzRo1hYEqpIA3C5rScxgwMB/6UfLsJ8KCwjYGAC5ijAG8lUAHibUA1cFhmAIjiIO4hUIPgRvoB3ybB/Rj/AvwoBPk3D/ANGP8C0Anyfhv6UemnZXQ8qAd8nQf0k/CKADs6D+kn4RQGZ0n6vDYOeREVWyFdBa9+FGDyPZBURFmOgax19WwAI09teTiW3NJeR5FWqpu0VotihiJHlYtbQnS+4d1aYqNNWQqSVDWPvc307kU20qe8zZnJ3bEvQWNPZONZfVzqB9lgbfiG6pRswtZw9ltW7/ALnSqbgH8tR51Y9hO6N4V6hhCgFoAoAoAoDFH0v3/wB1VLFfEdg/zjXmmw1BUEi0BBi9w8aArUIEpsAqQFQSLUkCVBIVJAtQSFBYSpBs9EsPnxackDOfYLD3kVaC1KvY9AmTMpXMy3FsymzDwPA13a0Ii7NO1/ExuikfVYeJGR1kl6yYhg5K5nzWkY7msyjU3NjXGissUut2b+Iy7WtKSaajljpbkraLpdPbRFLp+geKGNuuVDMJOujgOJjR4lLIs8ABMiMTutvUajSux5xy+HWaFsNK+GmjaODHYoQwQuY3nd8qeoQ3VM0RdihYWvYWJtQF3AYvabuYhLOEefCxido7lUEUk2JkUvCgs2WNBdbAnTvA2OjWFcbSxjSzTOUTDwIZEUK8axhzJmWNVJzySCwP2tN1gMDZztLh4sO0UiynHyYzGhopFSOOPESYg3cgK2bLCBYm9yeFAaHQrauNxMsDvNLIjwSyYmN4VjhiZ3VsMkTZAWbKWvqQVAOlxQHd1AOK/wDVbFZcIsYOsjj+3X41Etis3aL8DyvAxs91v6g9dh4Vjk1Fp/qeiPKo2gr/AKuS6d5PHJc+7u8BWzJFRtE1Ust7Igx0Q3j2+3jXKS9k44qmkrxKbr+XwtXNqxjtbRifz2VBBp4TEqFt1sif6V9ZfEX3eFSa6VaKjbM49yPRK9Y1hegC9ALQBQCUBij6X7/7qqWIMR2D/ONeabDUFQSLQEGMbQc71eFOU/dRxq16dJe27FnCejIgd2zv9jfYnhb4142Kp8SrVnRpQcY/u699+/uLLG4SNPO5XfTn6Gku0MMDYMo7wDl87a14r4VxKcFNwk13vX0vc0f7DCKWXOvp6mdtHEYZlOQfOX0IBAOut+Fe3w3h/E4VIqq7Q53af83MeI4phVFuGr9DKzV78sLNbamanxOjLR3QorM01oz0IyUldai0AUJEqALQBQHW9AsP9LJ/tQf/AGb9tdaaKSZ11dSoUAUAUAUAUBHiIVkRo5BmR1Ksp3FWFmB8QTQD40CgKosoAAA3ADQAUAtAeV/+ruLzTxRcFXMR3nd7jXKq+RjxkoqOWV9Tz4NbdXA8seJSOOlXzy6llUktLmjs+Vb2a7AixRsv5mrJabnoUJqWjbfcyTF4EdegUeo9jz7PaHkPfVJXb1KVqCdeNtpfTczsfFllcDcCbeG8fnUGWvHLUaXUhRrcBUHK9j1S9euesFAAoB1AFANoDGH0v3/3VUsQYjsH+ca802GoKqSV58TwXz+FbaGGv7U/Q8nGcRy+xS35v7FWt6SSsjxHJyd3qwqSAoAoAoQBrnUpRmtTTQxNSi7xenTkKDXmVKbpuzPosPiI1o5o+aHVyNAlAFALUg9C6I4fJhEPFyzn2mw9wFdoLQ5y3Ke2ZWlxIhgxDCZeruFfJFD62Zi+vzrsugSxtvNr3rlN3lZPX5ffwPVwsVTo9rVgnF33V3LSyS/ak9XLTzOkjcMAVIKncQQQfAitF7nlOLi7Pc4OHGtiQcRJtGSCVsa0EECG8aLFN1eSSAayFlR2Zm0UNfQChBKnTaYyMEhjdWw8k8KhnjL2lSKAZ5bB+sMi2so10GagGP0mxU5gERjV1lxTyg9fEvVYWEBlmR1zIRJKt1F75Ab8gNDaO25pMBgnjvFPjJcIoCm5QSkSy2JG7q1ffw76AdsHpRNiZoVaBFhnjxEsbLIzSCOGVUR3UqAA4dSLE/pQHU0AUB4d09xYlx8xvfKcg5WG78651EcKuqkm15nNj3cq4HjPR2EtUC5YwzENbeK7wzK3Q20lOFucX8DX2aVzrvDC+/VTcWNuRrpVWlzXSy5l1JtpYD1Zn4sFI7sts3nasxFehpOfVL4DdkYKNoVLKCTfXwJH6URGEpQdJNo7SvVAAUAtAJagFoAtQGKv0v3/AN1VLFfEdg/zjXmmwu4mSwsOP5VowtJTlmeyPO4jiXThkjvL5FSvTPnifD4OSTsISOe4eZrDiuJ4XDaVZpPpu/RGqhgq9bWEW112XqzRw+wHbew8FDMa8uXH3P8A9ejKXe9F9T06fBJ2vUml4a/Ytr0cXiX/ALB7q4S4nxOW1OEfG7+pqhwXD85yfp9h46OIdwc+BU/pULHcVeyg/Uu+D4Vc5eq+xFJ0cA4uD3hT7haun+14hD36MX4O3zucnwSi/dm/g/sVJthSDcynxup99do/5BBfnUpw8rr6GWpwOqvckn8ChiMFLHqyEd+8eYrdTx+DxkctOom+mz9GZoUcTg6meUHbnzVvIiBrM1Z2Z78ZKSTWzCoLC0ABSdBvOgHfwqQerYWERoqDcqqv4RatCVjkNODizZ+qTPe+fKua/O9r0yq97HTtqmXLmdul3YkhiVFCooVALBVACgDcABuFEraIrKTk80ndsrx7Lw6ytOuHiE7aNMI4xKwO/M4FzuHGpKldOj2BVCi4PDhGBUoIYgpUsGKkAajMqm3MDlQEsex8Kq5Fw0QTq3iyCNAvVSG8kdgOyxAJHGgJ3wcTZLxqerN4/VHqHKU9Xl6pI04GgG4fZ8MZUxxIpSPqkKqAViuD1a8luqm3cKAs0BHiZciM/wBlS3kL0B4ltDZLvGcSXTKbyalr2JuANN+4e2vMqcTpdt2FnmTty+5dYZxpOtJq1rmHjMLJGw61cpZQwU77EkC44HQ6V0hXhWvKLvZ2PCxFOpTazqzav5FYAm55a/z3Vds5WOm2ZsFpY1lVlysDdTcWNiOXBgax1eLUqNRwkndeFvme7hOHyqU1Ui1aXLv/AOwwmFKSmOTeNdOySDvB53/WvSVVVacakdn9SI03CbhLkaZF9Durmd2V9nQdXGEPAtbwzEg+VEcqFPs4Zel/mdNXqmYWgCgCgCgCgMVfpfv/ALqqWK+I7B/nGvNNhpwbNkla/ZX7R425DjUS4rRw8ezgnOp0jy8XsjyqvD62KruXuxWl39FzNvB7EjTUi7c21PsG4V51apjMV+bPJH9sPrL+o9PD8Nw9DVLM+r+i2NbD4UMwHvPKmGwNGEkoR8938TbOdlc2o4wososK9mMVFWRhlJt3ZFjApRifqgnvBAvUTgpKxMJZXckjCpZbgE7hcXYgXNudTGKirISk5O7FmiDizefEeFJRUlZkRk4u6Ml0sSDw0rz5Ryuxui8yuQtEOXlpWarhaNX34p/P1Lp2OP21EEncAWGhGgG9QTu771soQyU1G7duruZKiSkVVhYqWCkqN7WOUe2utytmMoQaPR7D9Ziol4Bsx8E9b9PfVoq7D2PRsQzBGKAFwpKgkhS1tASATa/IGu7vbQrBJySlsZnR3aEs6ymTKckmRWEckTGyKzZonYshBYjW1xY2rnSm5Xvy8jZjsPTouGS/tK+6lza0aST28noHSnacuFw/WQxF2zohOSSQRozWaZo09Zwo1yrqe7fXUwmNgOmqLh1fEMkzuZWjOFUgPh4ioaVklYdURmsULE3Gl9wAc/S13xAjwyo0Ty4KNHIe56+N8TO28dmAIRpoz633AC9jtsYmTES4fAxxFoERpZZzII88ilkiUILk5QCWvZQy6G9ARbM6aYeTC+kSBoysEM8iAM+QTsyRqpA9YlkIAtc3HOgL2wtt+lSYlREyrBOYQ7K4EhVRnIuANHzCwvoAeNAa9AZPSmfJhX5tZR7Tr7gaq3oSjyPamzmkULHOVmAYxw3YCXJdyqkaBgN3O1Y5pRefKmub5+Pf3/U9DB1IR/8AHNXT27jlBiXYjMzE3GpJJtpYXPtrt2cY+6kkdsVgozrSryV1GGnjrr5D3msSBwt7RyPPh5VChdJni4Hh3aUIz/8Apmj4aXT9Y/EubOkkOIjRZGCEg5QzBdLsdL91V7Om1dxTfgbeFTccFUi94tr1t9Tr2jBYNxAI87fCrLRWRxtd3H1AC9SDdr1DCF6AKAWgCgC9AYq/S/f/AHVUsV8R2D/ONeabDudkEGFTxtYnjoSKyZIwbUVY1Qd4ou0LljAGz+II/X9K7UHaZyrK8TSrcYyk2HihEjIiq0rZnsNXcqFzNz0A8q51J5Yl6cczJBBFK0crRq0sebI5ALxlxlfKTqtxobbxUwmpRuROOV2LV6uVMrFsC5t/LC1efWd5s20laJDeuZ0MiXZKzTtJIfmxYBR9YgAG55Xv/N91O0bI5uneV2TbZgZoerjsqkgHgAo1AAG7UCrUY5pEVdI6GB8jP9tf7q19j3mbMb3RHZZjlaRiDZMotfexGvkp86lU8ruQ5XOkx+DSZMj3AurAqbMrIwZWB5ggGplFSVmXo1ZUp5o9++1mrP4EWD2akRBRnvd2a7E9Y75bvJ9ojKAOAFQoJbF6mIlUVpJcktNkr6Lpvr1ItsbHXEmNutliliYtHNEyCRSylGHrqykFSRqDVzOZj9CcN6nVyTRlVdGZWRnlWWTrZM7SKxDFyTnXKwzGxGlgLkHRqBJxOGcuJZZspKlc8sKQcr2WNMqi/wBY3vQEGP6KpJNNKmKxEIxAQYiKNowkuRcgN2QtGStlJQi4A460BDL0LhMpZJpUhZsKxwq9V1J9Dy9SlypYL6o9UEa3NAamxNk+iiRRKzrJNJNZggKmVy7C6gFtWOpoDSoDnemCGRUiUgXJc3vbSwH5mocbqxKOE23sWYNhniIMiYqHKBe92bKN+m8rVHSaR0hNJ695wm10VMVMqdhZpAvgHIHurnR/Lj4H0VnKlZ819CpK12J76vFWRn4fRdLDQhLdL4vU6HoVg2nxGlh1cbHMd2pCgeRPlRQu9DHiqKoZ5Lao0/NXv9Gdx8jv9tffVux7zz8wfIz/AG1/uqOxfUjMHyM/2l/up2L6jMW63mMSgAUAooBaAQ0Bij6X7/7qqWK+J7B8K802nW7L29hZZjBAwLBc2gtGbdoKePA1TEZW00egsFXpUs81ZfE2aznMWgLAxjjke+2tdlXmkcnRiQyOW1JvXOUnJ3ZeMVFWQRSMu4/CkZuOwlFS3JnxbkcB4b66OvNooqMUVjXE6lTaG0Y4I3kkOiLmI4m+ijxJ0FLF6VOVWahHdnC9DelBWd0xDfNzOXDE+qkjG/sU7u6w766Sjoe7jsCnSTprWK9UegYtgI2J3BSSeVhe/uqsHaSPnZaxZy3+KMJ/UP4JPhXsZGedmR2PRWdJcP1sZurs1iQRopy7j3g1zkrOxZO5a2ltaLDmNXJLyuqIii7HMwXMRwUFhc9/OuUpqNr8zTQwtSspOO0U22+5X9dNi9VzMYE3S3DpKyGOYok64ZpxHeBZnyWQsDffIo3caA2DjItfnU9Xteuvq6ketrpqCPZQEW1NpxYaBp5mtEoBJALE5iFUKBvJLAC3OgINkbdgxTSJH1iyR5c8UsckMihr5WKuBobHUcqA0gw59/soBaAKA4zpRtyCHElJXIYIoACsdDc30Hf7qvGLZDdjn9q9KoBC5hf54C6Zo2tmGoOotfkeBtUVqTlBrbw3OuHcXVinzZ5czEkkm5JuTxJNcUklZH1AlSDrugO0IMOJWlezNkCgKx0GYncO8eVdacW7s8fik0nGPmdd/ijCf1D+CT4V0yM8nMg/xRhP6h/BJ8KZGMyE/wAT4T+ofwSfCmRjMi1XUzgBQC0AUAUAWoDCkmVJLuwAz8SB9aqljA2ztIMOrjPq/WPPurzZJp2Z9bwfAp2rz/4/f7epmYbEPE6yRsVdSCrDgRVD6CcYzi4y1TPUujnSNcUuhHXAevAbA6fWiPEdx3X3jjxcbHy+MwU8PK+8epvxTK3ZPiNxB5EHUVUxj6Axtp4vGCbJDAOpyBjPcszNnAKKgHq2W5ub34DfXanGDXtM5zcl7qKh2jjkK2gaUFwGW2UhS+UsGyKBYHNre9juq0oUraMopzvqjo2cAEkgAbydBWc7lTE40KhcsI4hvlbT8Knf7e6wNTYmEZTeWKu2eY9KekPpTZIgRh1NxftSNu6x/wBBwHu6xjY+lwOCVBZpe8/h3GEiXrvSpdoxxDiCwkE7Nt7dPNl3r3y5S7FfslmK+V7V6EYRirJHxFevOtNzluyMmrnI946O4IwYSCIjVY1zf7iLv/cTWWTu2dEUcV0dZsUs64h7dYJHQiMj1FYRohyXCgsdCfrEjU3rNKleea56dLHxjQdJwWzSevNq7eu/8LbQ2sNnyL1oUSWGYLcpfjlJ1I8a7K9tTz55czyXtyvucHs7obilnjkkTDq6YyTFtiUlneWUs8jCPqigRbhwpa5IA01qSpJB0NkgjwbLh8PPNGsrYlJDlWbESi6ys+QlwjNIACNA9xawoC5jejmIj2bhcJBklaGXDvKHYxq6RP1rKpsbDMFAFtB4UBW2j0exk0hxM8UcjSTQ9bgFkHVnDQJLljMjqA5MkgdgQFOUDhqBSfoViskUa5EikTGpiEjaywx4iePEJh4dBdSU6skAaMx00oCfDdFsTIYHxkQkbq55Zo8ylOtEKYfC4ffqBGZbnslmc8dQOs6LYF8PgsPDKbypDGr65vXCjML8bG49lAeQdNMQZsZMwbTNYeA3Vppr2TnJ6mDEjowYBWI4MAynxB31FSmqkHBtq/TRloTyyUlyLo2gfrYPDH/t2/I15z4WuVWa8/4Nq4jV7hTtDlgsMP8At3/WoXC+tWfr/BP+yq93xKmLd5SCY40twRVQa87b624fDKgmk279XcyVq8qrvIbFCwOracq0HK5PUlQoD0moOYtAFAU8TtWCPtSC/wBkes3kKE2MnE9KVH0UZPexsPIVJOUycTt3ESfXyjkgt79/voTYs9FdkDGTkSk9Wq5msfWbWwF+++/urPiarpwut2d6NNTlrsdZtDoPg5F+bVom4MpLD2qx191eTnZ9DS4lWhvquhwm3OjmIwhu65or2Eq6p3X4qfH310Ukz28NjadfRaPoZUUhUhlJVhqGBIYHmCN1WNbSaszrNm9N3Fhik6y2glQ5JgO+2jeGlUcOh5FfhMJa03b5HTYPpXhH3YrKeUqWt7RYe+qZWebPh2Ihyv4FwbVw7ajEYc94lyeYBNLHB4Wst4P0IJ9uYRe1iYfYzTH2AUsWjg68nZQZjbQ6cwL9CjTONzyWVAeYUfAHvqcrN1LhFR61HY4/a+2p8U15nuBuQaRr4L+p1q6ikezQwtOgrQXnzM+rGgUG1TGTi7o5VqMK0HCaumWVcW3jzqas3OeY+KqUHQm6T5P+/AXMOYrndlCcY2XhK/43+NLsWQ8bRn4Ty/8AJJ8aXYsh67VxI3Ymb/ll+NLvqLIkG28WP81P/wAsvxpml1IyroPHSDGj/Nzf8j/Gpzy6jKug9ek2OH+bl/EaZ5dRlXQeOlW0B/m5PMfCmeXUZI9B46XbRH+bfyjP7adpLqRkj0HDpntH/wCU34If/wA1PaS6jJHoJL072kgB9IvruMcVvctSqkupGSJzyYhpGZm3k3J7zcmt+Hquas+RnqwUdh9aDiFAFAFAFAFAFCTtMT0gw6bmLnko08zpUHPKzKxPSiQ/RoFHM+sf0FSWymTicfNJ25GI5XsvkNKE2K1AFAFAbHRbawwuIDvfq2GR7bwCQQwHGxHleuGIpdpCy3OtGpkldnpHythsub0iLKeOdPjXkOMouzR6lOEqmsFfwM/au38AYnWSaN0KkFFOcnuAHGoSZqpYTE504xafXY8kFdz6sKAKAKAbGoXQbgAPzqXqcqcFD2VyS+o6oOoUAtAFASRqCLEXFbcN7UWmfI/5BBRrxkt2tfJjvR0+yPKu/Yw6I8LPLqJ6Mn2RUdjT6DtJdQGGT7P51xxEIQhdLU60pSlLcgaIdZYbvE+NZKMc00jtUlaLZP6Mvf5tXofh6fQzdrIPRl5t+Jqj8NT6DtpB6OObfiNR+FpjtpDJorKSGa/iedefNZZNGuLukwhQlQc7effWqjh4TgmzhUquMrIJkKrcO3urJJWk0d1qrh1GYC7sePD4VtjhYNJ33M7rSTsSxR5a70qSpqyOU5uW4+upQKAKAKAKEhQgKEhQgKAKAKAKAKAKAgnGt6x4qOzPpv8AHayWek30a+v0I6xn04UAUAtAJQDUW19SeOveSaHKEVFtX7/VsdQ6gKAKAKLUiUlFXk7IsRLYV6VCDhGzPheK4uOJxDlH3VovuPrseaFAF6wYyXtKJqoLRsrYUXJY/wAvU4SOrkRXeiRZrcZwoAoCPE9k/wA415Nb8x+Jup+6hML2B7fzrfhvy0Za3vj5Vup8KxYhWqs0UneCG4c3UeVbsO700ZqqtJkldjmFAFAFAFAFAFAFCQoQFAFAFAFAFAFAIwuLVDV1ZloTcJKUXZooByNDwryJJxbT5H1mG4tLKu0V115jusHOh6UeIYeX6reIGZPtDzFTZnT8ZQ/fH1QCZPtDzFLMfjMP+9eoucc6gj8bh/3oZGQNCSe/jxo2cI4yhTVnPN8+YjYgUM9Xiy2pr1I+u/1D3UszBLH1pfr9Bc551Up+KrP9b9Sxg0u1+X5mteFheTl0PPxtaUopN3uWZJAu/wAuNbzzrEDYk8NPzqLk2IWcneaAtSerHb2ee+vJqSzTbNsFaNioK34eNoLvM1R3kFdygVACgLbfR+yvLrfmPxNlP3UVAa24b8sz1feJoZ8osRf21Fah2jvcU6mVWIgxG46VejTdNWuVnJSdxyMxOhNdiheFSQFCAoAoAoAoAoB0QBIDHKt9WsTYcTbjQGo+z4zZFusjWC3kRmzEXUSIACt+YJtcXqtybGSKsQFAFAFAFAFAUW2YpJLO5uftf+K5ulFu7OnaStZCHZEX+r8RqeziRnYfJEPI+dOzQzMT5Hi/1edOzQzsVdkxjczj2/8AiodKL3JVSS2FOyozvLHxY0VKKDqye7E+SIeR/Eans4kZ2HyRD9k/iNOziMzF+SYuGYfeNR2cRnZYw+GCbixHIm9WjBR2EpuW45oVOttfbVitxpwy99LC4noo5n3VFibg2HJ+sap2UOhOeXUb6Kefuq9iLiejHmKWFw9GPd76WFxPRm7qWFyTI+XLpauEsNCTuzoqzSsRjDN3V0hTUFZFZTzO7FGGPMe+r2K3Hphhx1/KliLkyi26pAtCAoAoSFCAoAoSFASYYXYepnA1Kai4G8XG6oINl5iPWWNy66qWWFbMAQrNINZMoBtu3VBYwyLactKsVEoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoAoSLG9jcb/AH3VBBN6U3Jd1uyvK1SSQsbknnrQCUICgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgChIUAUB//9k=',width=400,height=400)

<iframe width="803" height="535" src="https://www.youtube.com/embed/PvMYTHbF9o0" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

Hair Air Lyrics
#Welcome, sulfur dioxide/Hello, carbon monoxide!/The air, the air is everywhere/Breathe deep while you sleep, breathe deep
Bless you, alcohol bloodstream/Save me, nicotine lungsteen/Incense, incense is in the air/Breathe deep while you sleep, breathe deep/Cataclysmic, cataplasm/Fall out, atomic orgasm/Vapor and fume at the stone of my tomb/Breathing like a solemn perfume/Eating at the stone of my tomb/
#Welcome, sulfur dioxide/Hello, carbon monoxide!/The air, the air is everywhere/Breathe deep while you sleep, breathe deep
Deep, deep, dee-deep

Source: Musixmatch
Songwriters: James Rado / Gerome Ragni / Galt Mac Dermot
Air lyrics © Emi U Catalog Inc

In [ ]:
sns.distplot(dgfearn["year_of_capacity_data"].apply(lambda x: x**4))
plt.show()

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'https://earth.esa.int/image/image_gallery?uuid=85186525-113d-4bf0-a72c-0f6f7effd216&groupId=10174&t=1330427591162',width=400,height=400)

The Sentinel-5 mission focuses on monitoring of trace gas concentrations and aerosols in the atmosphere to support operational services covering air-quality near-real time applications, air-quality protocol monitoring and climate protocol monitoring. https://earth.esa.int/web/guest/missions/esa-future-missions/sentinel-5

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcReZ328Mtn7rSFhjzv0_yZ4aJ7PQDvTdj3a39V-GUMkyAMAN7Jg',width=400,height=400)

Image dreamstime.com

Nitrogen dioxide is the chemical compound with the formula NO
It's one of several nitrogen oxides. No is an intermediate in the industrial synthesis of nitric acid, millions of tons of which are produced each year which is used primarily in the production of fertilizers. At higher temperatures it is a reddish-brown gas that has a characteristic sharp, biting odor and is a prominent air pollutant. https://en.wikipedia.org/wiki/Nitrogen_dioxide

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'https://www.centreforcities.org/wp-content/uploads/2020/01/Figure-7-Share-of-PM2.5-NO2.png',width=400,height=400)

Image  Figure: NO2 and PM2.5 sector breakdown for UK cities - Holding our breath — How poor air quality blights cities -https://www.centreforcities.org/reader/cities-outlook-2020/air-quality-cities/

PM2.5 refers to atmospheric particulate matter (PM) that have a diameter of less than 2.5 micrometers, which is about 3% the diameter of a human hair.

In [ ]:
fig, ax =plt.subplots(figsize=(8,6))
sns.scatterplot(x='year_of_capacity_data', y='primary_fuel', data=dgfearn)
plt.xticks(rotation=90)
plt.yticks(rotation=45)
plt.show()

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTHfwTskFzVxCdF4dHB64cLoWQViWWJi_rtdAqWpxg1Oawe3A3J',width=400,height=400)

Image americanactionforum.org - PREPA (Puerto Rico Electric Power) must abandon its dependency on heavy oil for the sake of the Puerto Rican economy.

In [ ]:
#Code from Parul Pandey
# Let's check the different kinds of Power Plants based on primary Fuel used.
sns.barplot(x=dgfearn['primary_fuel'].value_counts().index,y=dgfearn['primary_fuel'].value_counts())

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'https://www.ncdc.noaa.gov/sites/default/files/20120201_GSI_Snow_Water_Equiv-small.png',width=400,height=400)

The Global Data Assimilation System (GDAS) is the system used by the National Center for Environmental Prediction (NCEP) Global Forecast System (GFS) model to place observations into a gridded model space for the purpose of starting, or initializing, weather forecasts with observed data. GDAS adds the following types of observations to a gridded, 3-D, model space: surface observations, balloon data, wind profiler data, aircraft reports, buoy observations, radar observations, and satellite observations.https://www.ncdc.noaa.gov/data-access/model-data/model-datasets/global-data-assimilation-system-gdas

In [ ]:
plt.figure(figsize=(10,10))
plt.title('GAS EMISSIONS')
ax=sns.heatmap(dgfearn.corr(),
               linewidth=2.6,
               annot=True,
               center=1)

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTlj9a038ATbo37-4ssx4JNwiKyezPp7M9_lyYsahzDYhTvw_Yt',width=400,height=400)

Image slideserve.com

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ_TXuwb5plSTxglEZduU80WnkM6BoJGw_kewRGaKaBOJ_TbAtV',width=400,height=400)

Image shutter.stock.com

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxITEBUTEhMWFhUWGBgYFxcXFxYYGBkYGhoYGB0XGBcaHSggHR0lHhgYIjIhJiorMC4uGR8zODMtNygtLisBCgoKDg0OGhAQGy0lHSUtLS0rLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0rLS0tLS0tLS0tLS0tLS0tLS0rLS03Lf/AABEIAQMAwgMBIgACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAAAAwQFBgcCAQj/xABREAACAQMCAwUFAwYKBQoHAAABAgMABBESIQUTMQYiQVFhBxRxgZEjMqEzQlJykrEVFiQ0YoKissHRNVNzs8JDY3R1lLTS0+HwFyUmVIOTo//EABkBAAMBAQEAAAAAAAAAAAAAAAABAgMEBf/EACIRAAICAQUAAwEBAAAAAAAAAAABAhESAyExQVEEMmEiE//aAAwDAQACEQMRAD8A2Wio6+41FE+hhMzYBIit7ibAOcajEjAE4OxpV+KRCETFsI2MHS2ok7BQmNRcnbRjOdsZrCjax5RUY3HIxguk0an8+SGRUH6zEYQerYFcHtHBqxifAONfu1zytjjPO5ejT/Szj1p0wtEtRUM/ae3G5E+n9MWt0YyPAiQRaSD4EHB2xXv8ZIf9Vd/9hvf/ACaKYrRMUUzuOKRJEspJZH06OWjyM+oahoRFLNtk7DoCfCmydoYCrEmVdOnuvBPHIxbOkJE6B3J0tgKD900qHZK0VGPxyNd5EmjX9OSGRUHq74wg9WwKeX14kSa31YyBhUd2JOwCogLMfQCigsXoqG/jPb+InDeCG1uhI3XdYzFqYDByQCB411/GW3xtzi2SDGttctKpUITrhEetNpEOWAyHXzFOmFol6KYni8XJ5+WKE4AEchfVq0FOUF16gwIK6cgg7bU1/jLD/qrv/sN7/wCTRTC0TFFQq9qLckhVuWIOGC2d42lsA6XxF3WwQcHBwQfGiLtVasAymUocYl93ueTg/nc7l8vT/S1Y9aKYWiaoqGHaaA7hLojwK2V6QR5giHBHqKkrO7SVBIhOk56qykFSVIZWAKkEEEEAgilTCxeioZe08BGVS6YHcMtleMpB6FWEWCD5jalZu0EC6cc2TWiyDkwTzdxs6WblI2kHBxnGcHyp0wtEpRUOvaa3P3RO+DhuXa3Umlh1R9ER0OPFGwRncU5tOMwyOEBkRmzpWWGaEtgZOjmouogAnAztRTC0P6Kjm4zHqZUWWQqcMY4ndQwJBXWBpLAgggEkeOKcWN9HMpMZPdOllKsrq2AdLowDKcEHBHQg+NKgsc0UUUwGfCv51c/CH+61RPAU1TWudwkV24Hk/OjQMPUKzr8HbzqW4V/Orn4Q/wB1qh+z02LuKF0dHWC6OGUgEc+HdT0I7w6edaLgzfY/4FfSTXfEIpCGjiljRFIXAVreJ2U7b5ZmO+evlUN2bhL8CntxkFFvIV8SFDSiPrnfQUNSfZiFhfcTJUgNPCVJBAIFtCMg+O9JdiFXROmciTlz9fzJYxGMeh5LH5mqJGfaHMXBLeMfe/kKfECSEt/ZVqnu0l3cRiPkdCzcxhC0xVQpIwqupG+N9/hUZ24i+yt4wfuMXPqqRMn96RDUl2meYcoRNKqlmEjQxpI2NJwMOrAAnxxQAx5KIvDUjbUiyIqMfFRazBSfiADXFz/p6LV933NwuenM5uR/W0CbHoHpR2jI4dyvyfNXRnOdPu0+M53zjHWmnHoGn4oLbOhZLXmGQDLK8UpC6DkFW+0Y6huNG1JKipci91xie3uZo7mJ5YZnX3d15CxKpRE5Ds7rhy+rAb72oAHOw6jtnitrGOU5kR7ZHOc5ZUwTnx3B3pbgMkkpu7O6XnJCUjEroAJ0kiViHXAQkEkHTtgjbOabW7k21lqYsRLCNROS2nUuonxJxnPrQ1YLsOK/6bs/9jP+6pAcLZeKe8DOh7ZkYeAdZIyG+LKQPhGKj+Lf6bs/9jP+6pnhHFBLNdRfnW8oT5NFHKD/AGyP6tMkiOFWhktmCtpZbu5dWK6gGW7mIyuRkfMU7fikw4a0+U5yq2+g6NSsVzo1ZxtnGr5152Y/Iv8A9JvP+9TU1m/0RJ8JP961Srtly6HXZaFla9DsHc3OSwXSCTb252XJx9TUIvD5bfs2IJl0yR24V1ypwQemVJB+RqxcCP2t7/0gf92tqrx4lJc9nOfMQZJLcMxAABJPkNhTXBPZNS3E6cPga3GX0wA/ZmTCHSGIQMpOFz4/WmkkvK4ZO8bF2CXMm66CZWaVmXSfu4kJXBO2Opp1PJMvD4DCXDabcExoruEOnUVVgQds+BppMV9yVIy7a7iNH5g0uWe5XnalwADvIcAAeW1TIcSfhmSJ4rYdTE7L+rEYkP8AvFqvdiyfeblSMctUjA/opdX6p9UCn51MXdvEb6CRpgsqxTokRK5dXMTOwBOTp5adPOovs6jLxXiQPQi0dP1WSXP9sOfnVkifs768Q/6wuP8Agry4luX4TG1wNN2WiwMBSJeeoTYdM7ZA8C3rXvs768Q/6wuP+CmnDrhrng9vczENOrK6ykKGDCbQSCBtlMqcdQSPGkA2PHyI7S2tg0ZintUlJltmzHzEjdSEmdzqLDO3mSas91heIJgflLeQt68qSIJ9Oc/1qE4fxGP3WxT3J5GAh0l41QBkhZg8bv3dXdONx161N2ySyTmeVBHhDHEmoMwDFWdnI7oJKoAozjRnJ1YVSexUeSQor2vKzNBjLYvzGkimMZcKGGhWB05wRncbH8KbT8ELyLNJO5mjBEUiqilASC22CGDYAIbI288GpeithUiMmsrmRSktyNB2Ihi5TlfFTIXbGR4oFPkRXbcM0ya4H5R5aREBFZdEZcoAD0xzH6edSFFAUiJueDGUlppmc8t41wqqFDmNi2AMFsxrjPTfzpzyLn/7o/8A6o6e0UBiiO/ggCGGNHZTAVKPhScqjJlhjByGb60lJwUu5keZ+aAoSRQilNOvoMEHPMYEMCCMbbVLVxNMqKWYgKoJJOwAG+aApEfNBclSJLtVjxuY4hG+nxzIzsBt4hQR4EVUO0Pbrh8MaQwMzmEoU5IXQnL2A1N3SMDGBmqP7Qe2D3khRSRbIcIg21n9Nx4+gPT471SnJyfwFQ5hSL3f+0mVp0nVftYwVRmKkAMDqUoEGc5G+fAetc2HtLmjkklEWJpcc1wVCvpAC90qcaRkbHxOaomjHpXDeQGam2GKNn7K+0y1VeXOjRd52L5DjVI7SMSAAQNTnoDgVb7S2SeHTFd67diTpURnusxYrrxnqSPOvnGO0fTkKad8K4tPZzLLEzI4GM4GDnwYEYIoyY6PpE2Dh5GimKCVg7LoRu8ESPIJ6d1F2+NMLbszotVsxO5t1UJoKpq0A50l8fLOKYdjO3UF4qI5CXBzlPAkeKnxyN8VbqWTFSI63sZ0RUW5OlQFXMUZOAMDJrwcJ+zC8xtYlM2vC/fJYnu4xjvHb4VJUUZMKRDzcFZ5UnadjLH9xgiAAFXUgrjfIf8Asr8/Y+EyrM84uW5kiRo32ceNMZkZcDHX7Vsn4eVS9FGTDFEPw/gjQczkzsvNkaWTKI2ZHxqYbbA4G1JWPZzlW6WyzuYFI7hVNRAfXguB5+nSp2ijJhiiLtOEMghVpmdIMctSqDojRjUwG/dY+W9SlFFJuxpUFFFFMAoor3NbAeUUUUAFFFFABWQ+1HtS0kzWkZxFH+UIP3364P8ARXbbzz5VpvaS/EFpNLnGhGIPkcbfjivnzhVmbidIsklz3j1IXqWPn4/Opk9hi/AuzU12dSaVQHd2B6+QxuflVqj9m6eMxz5hQPoCTVvs4EjQIgCquwAFP0PlXM5M1xSW/JSW9m8A6ySH9n/KncHZO1hG0QPqdzVt1edIXCjFIcErK5PbqBsu1V7itojghgCD6fjVuu4u6fSq1eCpbo0aTKBPE8LbEgqdiNjX0H2A4t7zYROW1Mo0OfHUoHX5YrGOOwa0zjcVPexvi5ju2tye5Muw8A6DOQPUAj6VtF2rOaSpm1UUUU2qJCiiigAooooAKKKKACiiimAUUUVsAUUUUAFFFFAFS9qhP8Fy4OO9F9OYtZj2CjHNdj1wAP8AH91a529sjLw24UdQmv8AYIf/AIayTsA2XcDwA/fWOqVH7I0VZKco9MQN6998RPvOo+LL/nXOnZvNdknjekph1/Cmy8UjbYOCfQ5rt7imxwXY0u1ODVcuYOtTHF+JpGCXPwHjVFvO0cjtiJdumev40krobdEjcQbYPQ+lRHZd+RxS3JOBzVX9ruZP7QPyriHi8uQJcEfSn3E7UExSL4OmD8SMVorToxnT3N9ryiiumrRkFFFFTggCiiiswCiiigAooopgFFFFbAFFFFABRRRQAleQ643T9JWX6gj/ABrEuwlqUabVs6sFYfDr+NbdczBEZz0UE/SstZVTiU7KulLhVlG+Rr6Pg/Hf51jqlw2dnXG7+QARx/ePU+lV6bgSldc9wEPzJ/fVsktQzFh1IA3qCu+DF2cuWztoI/NIOcjPiawTSN521sRHDrJEcGKUuB6eHrg+njWgcF7671X+G8KCKExnvaix65xjrVo4ZAEJx40TS6CF1uZ72w184qegNMrvhUsIjIOdahgVGRv+aSD1zgeFXLtTEFlWQj0J6V00Sso22xnzoUsQcLKXJa4LIwJ9evz9KeMCbU46rjHxBFS09sAelMLtMRsB8aanZDjRovs0LmzLvK0jNK5JZtRBGBjPr1+BFWyqD7Hb1XtZo8AMkuo48QygA/2CPlV+rqi7VmL5YUUUVQgoooqWlQBRRRWQBRRRTAKKKK2AKKKKACiiigAIrL+09oYbnCDuiQMAPBXHeA8h129BWo1V+0driTmkZUgA589/8P3VjrLbYvT+1ELav61IRlfKoZXVDgdPj4U8SXO4rjOkesi5yB/76VxEw1ZJwBXKHO1NuKcCaZcByi9DjIPyIrVAd9qJojC2s+G3xxVe4Lch0wDnG1e9oeyLmLQsrEAZ72ST8TUR2ftJYXIc93p86mSEiVvGw2P3U15RZW2/NNdcRbHSluCnJGfhSgm2EuC1+zDgnIhklK4MxTH6qA7/AAJZquteIoAAAwB0A8K9rujxsct3uFFFFUIKKKKmT2AKKKKyAKKKKYBRRRWwBRRRQAUUUUAFJ3EKupVgCD4GlK9pNWBS+0nB1ijDxgjvd7qdj4/XA+dQFvcH/wBmtJ4jaCWJkP5wI/yrK5w0cjRuMMpx4/UelcmrBpm+lK9mWG1nORTyXi8UY1O4H4k/AVXbfL7asCvbq0RO8i4P6W5PwzWcWaji+7aRLq+ykZiO6ukjPxql3PaNnf8AJMmTjHWpK8vZmOyE7eP+VIW1k7Nqk8PDHSrG0lwKyylgM+lSvZ2HVKi+bAf5/gDVfu3xJ1pxw3jwjuodJ2R1LnPXfBHyBNOEdzLUe1G4CihTtRXWjnCiiimAUUUUAFFFFZyVMAooopUvQCiiitQCiiigAooooAKKKKACqd7QuGgqkwHeBKnH1GfofrVxphx625lvIvjjI+I3qNT6ji6aZlNjdkHB8KstlMjDfB+OKql9b57y9ajH4rKm2PpXGonWXu6niUdBVc4nxQHOOnnVaueOO3hUbc3zHatMWZvUoc33Ev0ev40xtJMNSFCtg1aMZSbPpDslxQT2kTZ30hW+I2NTNYF2W7XyWTDA1xN95CfH9JT4HH+FadD7SLAqhaRlLdQY3Ok+pAxVxe5LdFvopK0uo5UDxuroejKQQfmKVrRMSaYUUUUnxaGFFFFZN2AUUUUwCiiitgCiiigAooooAKKKKACg1xNKqqWYhVHUk4A+JrOe1XtK0ZS0XJH/ACjdD+qu341E30RLUjHkZ8athHPIg2AY4+B3H4VW+IwfOpOC+eZFlkcszrlifPy9MUnPHkGuU7NN5QTKxNB1qOmixVgubfeoudOo8PWrixOBGV5TkxVzyKoxxOAcjFKD7v0rpIT4D50jcHG1My1f5id8P4hNA5aCV4mPUoxXPxx1+dW/hntE4lHgGVZB/wA5Gp/FdJqn2kG2TTxSBQcbk1wabw/2pt/y1sPjG+P7LD/iq18M7aWU2AJeWx/Nl7n4nu/jWHoaVpp0C1ZI+iwcjI3B8aKwXhPHbm2OYZWUfo5yh/qnI+Y3q+cD9pKMVS6TQTtzEyU+a9R+NJnQvkR7L9RTJeMWxAIuIsHf8on+dFM0zj6PaKKK2LCiiigAoophxni8NtHrmbA8AN2Y+Sjx/wAKTdCbS3ZIYqu9pu1kNqCv35fBB4erHw+HWqVx3t7NLlYvsUP6Jy5+LeHy+tU2WUsckknz8fjWcpWcep8npEvxjtFPcn7WQkZ2UbKPgv8Aid6rt6ufnTiuZFyMVJgm3uyU7LTa49HipO3ocEfjmpmSOqVazvC4ZDhvwNWi17SRMAJFKHx2yPqKiUXyel8f5MccZCd0MZ2phJEDU7PPbsmRIPxz+6mpaHH31I+tTTOv/SDXJBizz0pSLhbscAY9dqkzxGBOgLfLA/Go3iHHn6LhfRc5+ZpqLMJ6+kuNxHiarCNIILenh/61CQx6mzS/LZzlqdJGAMVaVHDq6uTEicCm8jUtIabUzKI7tX8Kc5NMbU71It0oFLZgjV1SUZ3pWmI9zRTCS9IJGnOD1ooKPpmiioG97WwRFtSTlFflGRYXaPmagnLDDq2ohenWtj0bJ6iq+O1sfM5Xu93zdOvl+7vq0Z06/LGduua8/jamjWLW9KldQYW0hBUjII05O9AskO+0PaCG0j1SHLH7iD7zH/AetYzxrjEtzKZJW69B4KudlFKzz3fEZZJ4YJZ1zjVGo0KOoQMxAJA6gZxnfrUUqyGOSUQymOElZm5bYiYdVcHcEZGRjasnbOKcpT6PaKWawuBMsHu03OddaxaO+ybjWPALkdSRXN3aywvy54ZIXI1BZBjK9MqQSrDzwdsjNKmYYO90J0V1FDI4kMcUkgiUPIUUsEU6sFv2G+hpSCxuH5ei2nbmoZYsJ9+MaMuu/Tvp+0KKZpi/BuyA9abNEwOxzUvFwW9YZWyuWGSMiPxBII6+BBFeRcIu2GVs7lhqZSRET3lYqw2PgwI+VOmGEvCKDyDoBXolk8QKc2VlPPI0cEEssiffRUw0e5GJNeNByDsd9j5V6tnKZvdxBN7wOsGg8wbZ1EHbT/Tzp360qYv8/wAGhRj1NdLbAdRT2Xh9wkyQPbTLPICUiKZZwAWJQglWwAScHaljwi7EgjNpcCRlZwnL3KqVDMN+gLqP6wopjwfgyC0kxpdVflpKY5FikYokjIQrOuvKgnxGh9j+iaTgtpJZBFDG8shBIRBk6R1Y+AG4GSQMkDxooWL4GUlNGzmpaXgt2Zzb+6T84LrMejvaMgawc6WXJAyCdzikoeAXsiq0dpOwcFlIT7wGASATnYkfWimaRi6GkLYIqTj9aYxcOuNEUgt5tEr6Im0Hvv3u4o6k91vDwNSt5w+5gKC4tpojKwSMMoOtycBFKkjUT4Eg0UyZRd8DQDvV1dSaUJHXoPiacS8HvkdQ1ncKZG0RgoO82ln0jfrpVj8jTO9tbjQ7tbyrHA+iVimBHIdOFb176ftCimLBnKSAADA2HlRUSIJTuMYO9FBWJ9WVVbGz5ovYP9XxGCUfq5s7o/U66tVQLx3UF1NLBAkyTiIkGblFXRShONDZBUJ9DWilZ2yWw9u2VZJ7sHZLZlz/ALN5SfxFNOzpxb8O+1YH3QYiGftcRQ7k5xlfX9L41Cpw/iX8Hy2LwwsZluFM/vBwouGkY/ZmPJ0CQjrvpHTNQNx2zltJreJ7UM1lGYWKzjDkpGuoZj2+50PnTyRm9uSw9lpWHD7MgGMvfT60B6ZuLomNiuxwRjy7tP7Czjll4vDJgJLMkbeH5Szt1+pLfU1QOA9v+THyp7QyhZ5Z4THKFKmSR5NL6sZwZG7w6g/d23ZN29uAt3mBebczxzoyyYWPlCEIhGnLYEKgttnJ2FGSJzj6atbx44vJnqtjCM//AJp8/uH0rNO2b3fuPDDc8h1AUJNFNJK8p5B7z6o1GGxqJDNuB8aWHtLk/hD3oWuIzAsLxc0Fjh3cOraQMjURpPXPUU3vO2EEsdvA/DFa3t8hInmDd0JoTOVPeA9T8TSbQnKLVWTHsZK87iGrGnl22c9MZuc59KtkFgIOIcOgU5WKxuYwT5IbJR+6sg7P8VktLa8iCa2urdYQ/M08tlWZdf3SW/Kg+H3atD+0aRrqC5a0GYYZYmAmHeaUwtqHc2A5R2/pDyoTVCjKKXJJdj+MXBt+OZlb+TyXPJ6fZ45zDT8wDUn7OblpOD2kks7K73MhZ9yZGNzL3DjH3jt5Vn3B+0T28fEU5Gv39pWzzMcrmcwYPd72NY3GM4pzwHta9tY29p7vr5E3N5nNC6hzml06dBwe9jr4U7Q84+l4MziHjcgVonE5xggMAtrbhWDKdsjvDfbV51OpGP4ZkbHe9yjGfHHOlOKzW37dHm3hmtNdveMrGJZQJEIijiPewAwYRg9VI9a7T2hTjiLXjW45TRCDkB++EVi4k5mNOvLHK9MEb7ZJkh5xJ7hDXhl4Ibn3d1GsJNHNJLJJmym7zaowO8BkkMd/rTvhMn/1HKPfOeBbT/Y4x7t9ra5TOTnPXw6VU/49gNZCKy0RWbtoTn6iYzBJAqklTggOCdz06nrXsXbeNL/32Phyo7RSxy4mXVIZGgZWZtH5vKYf1/Si0GcfSZ9qtkkPD7VYzlDfPIMdPtlupjj0y5qP9khw3EmGzCCDB8R/OTsfiB9Ki4+1aS29ha39rzIrU5kYMZObot5okHL05yWZDknGRuQN6juxXaF7B5C0PNjnjWOWMPpYadRUox2OA7gg4zscjG6tXZLayTs1Azv7/wAGI7xktbkSEnfTyrd8k+PfVP2qj+23EzZ8Y4RpGmA86IgdDzTFHv6KWjb61XP/AIjFbuGZbPEMEDwRRmUczMhhJdmwRsIQoG/UnO+Kr/H+2j3MFmksGJbWdZuYJMh1Uk8sDTkZGjf+jTyRWS9NMurpE47YWKDuwwXExG2NUgZQfiAJPk9NeLNdmLvCCS3HE4yJOdI0yfy9AE5Zj0jB7uNey/Ss8vu3rtxhOJxwBdCCMxM+dS4cN3wvdPfBGx3UVLcT9p0TxmKGyZENxFcktMCxdbhbiTIwcaipAwds9PCjJDyRdO0z/wDz2wX3zI5mfc8Du/YXP2+rPjnTjHjXftQsUThHEHQ/l5IXf9dZLeE/hEPmDVJv/aLBNdwXQ4cFnhfJk5q62TlyJy86NhmQH5Ulc9v+bby2s9t9lLdLMx5moJAblJ3RlCZbo47vgelFoLRSTd42zRS3EuL2hmkMNgvKLvy8tpOjUdOV8NsbeFeVGP6Rj+n1BRRRUHUeisA43PzLmZz+dK5+Wo4rfXbAJ8gTXzoXyc+e/wBaDl+Ryia7HdlRxCaZTO0KwJE2VVWyZDIMHV0AEf4054b2C519fWsl06LZiEiQRoS4lRnyw6bAeHrU37MYQLHiErSLEDMicxvuqI44236bZkb61b4rfl8Tv5NIOu0tWORsSpulwfPZV+tapKgjBUjOD7PH98toYroPDcxPMJjHpkVI+XkBM4JbmpgnGMnION/OL9hyiwSWVwZkmuDbHnIFMbh3jL90LqVWjYEYydsHer9aSauI8NlJ082wnxGMCNTm0bEY6jY+Z2QeVRX8CwRi1uTFJDK3ESpDzT6STPMocRO2gczZxhfz9uuaeKKwj4VvtN2HFva3U1tdtLJaflkljVV/JpKdBUAghHVurDw67h3P7NXW9W396flNbyzCTlpq1xyRKUI6aSJQfPY1Yu1UbJZcdZwVV/uFgQGHudumVJ6jUCu3iCKuBvUa4lhx344kfP8AQlMi7fOE/hRSDCPhhHYfgzcRmEZlMIFuZndVVt9UahcNsAdTn+rU1b9hWPFXsDcsFS3W4WXlpqbLhNJXpjOrceleexeANb30hcRhbWGPmHomUmdmPoAUNaHyMcXinGG18PlXUOh5c0LD5HmmkkiYwVGbW/YCRhZc26kR7iaaGRVVQEMSTsSh6nJhxv4MalLzsTb20bThorpIZUinSSEhxqZFJVy2MrzFboQRkDBq5X92k0vCJo/uSzPIvwks7l/r3qju1kbRcN4m7qQGuVdM7alzbAEfEqRTotJIgV7I2k6ySsI7aBJ1t05UEbSPIZEi1MzKcAyOFAA2xknfAYQ+z/VxGSz5phEUXOM0YyHV2CoGjclUI0y5xsdIIxnAmeDh7vgkoh1pI/EFOUGt4830UmvGPzVOvywM9KkeGcEkhfifOvjKxtY094lVRyu7cMdQUgaVDq/h1phSZU7PsGzcUnsGuWCxQpMkojQs4chcMp22bV09K9bsDmezEd4Xt7sORLygsilYzKuFOxDAeIBGPXbR47XHGRMDkSWOnI6HlzBs/wD9R+FRVs/Nl4LPsgZZRykAWJdVszZVeoxpwN9gTSpE4R8KN2l7CNBAJba694xOtu6yJy8OziIEMvXDsoO24OQdt1+O+y9UguDFePJc28ImdGjVY2BDnC43UnltjvHGBnrmrp2uj1WfMlXk8riEDKFyiuvvcS65B0bIYtk+IBp7xpSrcTkYFUNlHhyMKdK3WrDdNsjPlkUYoeKM5h9lGp7T+Uycu4iZ3bQmUcIjqo8CCC/X9Gs2uhokdOuh3TPnpYrn54r6g4DeJotoTjX7tHKnngKqMR8Naj+tXzBxP+cT/wC2m/3jVM0qFNKgtpN6VkfuM3nsKZBsdKcMRhAemMmszMcx2g0jp0HlRSBeU7gbH40Uxn1bRRRSOwSvDiJ/1G/ca+bFvo/umRM7DGoZ+lfSV4uYnA8Ub9xqn9lVjWw4OnIhYXCrHIWjUnAtZZcg+eqNeudifjVRVmGrDJmPzXhCNBz3CSbtCJmVHzjJMYOD0H0rhu0lyXI9+uCxGk/ymQkgZ7p73Tc7eprUb3gdunC+IxJGqqLwouFGVVpIO6p6gDUQMdKsnEeG28hueHtBF7tHZwuihACrO1yuQfDHKQgjcHJzV4/pCg/TBl4uxMTNdyFodoWM7ExZAGIyW7uQoGB1AxTm74y84YT3kko1BmV5zp1DADaAQoIwMEAVsnD44DwuHibQoZI+HuxAjXDEpHIxxj9KLb0Y+dR3CLKL3bs/9mne06u6ve/kUx32333ox/QwfpmV5xd7mLlzXcs8a/mPNqX4tv3seBbOKLTik8twvJvJmnkAiDC5kLFclsEhs6FyzY+PnWrdqLKCe0uy0EIa2u4kiZUAYAG3Ykn15jKfAile2cEi3lqiwRJal0POVV5vPAlIiwCMIVA3weuPgYv0MH6VPifB7KzsZYUuWjfksGRblo+a2gjLxBwGz06dMCqceOSoVHv06lV0qPeZAVU6e6Bq2HdXb0FbN2Lhha2VpY0Zrm5usFlUnSJJtO5HTRGuPlUb7OOENHwmNEhikkS6eNzIqn7NbopIcnHeEYbHqBt4U2inG+GZOvE2jESi8lRYt4R7w4EexT7Matu6zDbwJFKXnFJJEHPu5pI8ggSzu8efA4ZtJI8K1vgHD4Vm4uLS3hZklXlRsihOb7sh0b40qZM56dTVS7B2Mz8Z13MCLqN2GQBTGJVKBlQZOw339amn6Q9N8WVGw4u8LMbe6eEuMsI5AAw/SKHKk4/OxnHjXC37lXHvcgjnJEzNcyaJCwCEykE5yMKc529K1vhHZ2K5srtOWmpeIXLqdK5zFdlwucdCF0/A1SfaHee6caknijiPIjjcRuv2bEwyIcqMeDZ+IFFV2T/m49hZWkxZGbjI7oxhb2QkR5UsoOoYBCj6Cp4WNl9mI72QaPyIF9J3Nip5f2m2xI28Kme2N+kV/wANtFt4NFzIGdtA1DltGwC42wehyDSna3gKxWPF5DHEBIjPFhVyAttHGfDY60Y7ehqzZGW9t5DzAhvXuYW6RvctKFcDxXUc7bgnODmoe/4/cPFyJryVosfknmOCB0DZ7zD0Ykbela/7WrCTkLFFBCtvIUEkwVeYkvNTlKoyNmbAJwcA0j7HrSFeHo8sas1zdSgFlUnuoVxuPKA7etTW4sdzITx64VlYXk4dEKIee4ZUOCUXvbKdK7f0R5VEvMpb74Ykkk6skknJJ8yetbDwrhccfB+JIY1zHfyopKjIVZYQAD5Yq69q+GQcm794t4Barbag+hdYlHN17jcaQIiCADknfyWIsT5ppeJMuPLx+FNo86RnrgZqV4Nwuab8lG7Z8QCR9azMzprnfbOPCvasI7JXvhbbeHeT/wAVFOh0fQNFFFI7DxhkEVm3Bu1PDoLfh8N1O8dxYbPH7vcN9oIZIGUsIyCBzCcg74FaVWO+0bgEgvJJY0JV8McbkHG5x13xVRdGWq2txW/7cWrWN8il+dNcmWGJo5QXUPCVJfSUXIQnc7eNTXEPaDw1efdpOzSzW0cK23KkEgeMzsNRIwMmbBPQacgnIrKJCOh600m267jzp5M51qs13hftD4ZFFa2nOZrdLQxStyLjZ1EKIpHLycrzdwMfWo+07Z8PSPhEfPZvdDiYiC4GALWWLVjl75Zl2GetZeux26GnMZ8qMxvUa6NQ4/224b7tcJbzSSvcXEcpHInUKA0IY5ZBsFiz4knbFLce4zZ3V3Dd2k00kqhIxE1vMsOjU5aXVJEArhXbvahnAG+ay0vWidkZAbWHP6OPoSP8KakVCeRLdnu1dvDbcPiaKV2iQe8P7tckwPymDMpERDEyHR3c7MT0qPue2PDltzCkzlv4RS4I93uRhPfVuHO8f5q6tuu2OtWeFMDasb40StxMP+dkz+2acnQ5yxVo0az7W8Pml4opmmVLrQFeO3ui+n3ZIi4xESpDA4yPCveASpbNbzHnNbxPcxrJ7vLzGV1QrJJEkesFnEneKjOAfHevezOcc2YZ3KIR8iQf7wrRWnxTTtGmn/SsrnCu06wRsBHcam4hPMV92uRm2llkJkzy8bI+rT97IxjO1U72nYuJp7uHWYpFSJdcckZLLGTssiq3U4zjwrSmuGPWqr7QJR7rgncuun5ZJ/DNKT2CcUojXtb2vsp+KcMuIpWaK2LGZuVMNGSmO6yBm6H7oPSpXi/aiCe14qkZnf3oH3bFtdYce7RR90mPCjmK43x4nxzWWgVrfZJ8WMIO3cB+uT+4inF2ZQnkI9s+L2t3LDLbSztPEyKsLW1wsTq0qa2cyRAAqmpgcjBHj0Lfgvbaws7W0t5Bqlindpg0MxNuZOe7SKeWQxBkCd0nIkONqlpm71ZP2n/n9x+uv+7SpcqZpP8AkvXFe23Dja8QjjmYtPcrLGOTcDUNNtqO8e3eSTr5eoqRu/aJww3V1IJyVks4olPIuN3V7klccvPSRdztv8ax5uteUszPM741b2qNGLOaSZOUhkMiFCsuWDIMouQAF3369atPY/jNyECwwhtIwO9gZ9RkVSpOlSPAuLywEhMYO+CM/SpvcnI0rn8ZO/LgGd8ZH/joqqfxuuv0/wCyv+VFVaHk/TfdZo1miioOsNZqj9qLlxxBFz3TEMjbwLUUUyJA9nG5w8aN+sob99NJ+zFmQfsF+WR+40UVT4RMlsRM3Z+2QZWJfnk/gTWd6iGYDwJ/fRRUnNLg7VjWidlP5jH8X/vtRRQVo9lmhlOkb1lXan+czfr5+oBP4k0UVcy9Tg77JTMt3DpONRKn1UqTj6gfStXZzRRVR4L0PqJP1qje0SQ64lztpJx65Iz9KKKyfJep9WVE/dNa1Z92NFXYBQAPQDA/dRRWkODm0uWKVlPaE5vLj/aH9wFFFRLk31uCLbrS9p0bYHABGQD4jzoopIwXJ60nTZeh/MU+BPl61zG24OF6for/AEvSiiqKFw/ov7K/5V7RRQM//9k=',width=400,height=400)

Image quickmeme.com

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMTEhUTExMVFRUXGBcXGBgYGBofHhgYFxgYFx0aGhodHSggHRolIBgaITEhJSkrLi4uGB8zODMtNygtLisBCgoKBQUFDgUFDisZExkrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrK//AABEIAP0AxwMBIgACEQEDEQH/xAAcAAAABwEBAAAAAAAAAAAAAAABAgMEBQYHAAj/xABGEAACAQMCAwUEBggFAgUFAAABAgMABBESIQUGMRMiQVFhB3GBkRQjMqGxwRUzQlJictHwJDSCsuFz8RYlY4OSNUNTosL/xAAUAQEAAAAAAAAAAAAAAAAAAAAA/8QAFBEBAAAAAAAAAAAAAAAAAAAAAP/aAAwDAQACEQMRAD8Ar9xbUzkt6mLgb4pnOlBcrCM/oGSNVLNIwiQDGWeWZI1G5AHeYbmq3y/yfxCC4ieSxmwG8GhPgT/+X0NWgRj9AzDHqPRhIpBHqCAc1LcElY3nB8knPDWJ3O50wbnzO539aCi81cKurm/kjitJ9ehZNDdmp0DCagS+kjO2xJqHi4FddnNL9Fk0QGRZmzFhDCMuD9Zk6R5A+ma1fgmDNAypiFuE/Vwgkso1x5XtOrEgqAcA7etVS2t//Ib5RZ3Jzcz6YAZe0iLqpUyba20ZBbUMHxyKCrT8JuIokmltpo4nKhXZRjL/AGdQBLLnIA1AbkClLCzklcRQxPLIQW0oBsoIGokkKoyQNyN60TnW0uJuHsZTonQWrSCDU8N19cuhU1IGL6hkKoyCyjLZqB9nJ1HiSYbX9FKBCpDakadGXQRqyGwCMdcUFcHD7g6h9Fmysy27AhQRK+kqhBYHfWpDfZwwOaUHBbssqi1lJcyKoDQnLQkrIMiTAKkEHJG4NatZOE4lfu7DsmaxhUdf8RpJHuYB4/mPIVSeWLyWPhfEw51XFrLcRo5Iysk+I3Pp9Zrf/VQVI8Pn+j/S/o05t9JbtdIxp/ewDq0fxY043zjer/zk00ljBaC1nE0hjWMERYYxoXYA9pj7Kk746U6SzaThcLJI6KODgEqqFW+pXKMWU4O3hg/ap1x2MfpnhrdjICBjtiZOzYNBdnswMdnrXTqOSGwwwCAcBjdzw+aNEeSJ0WQuEZtPeMbaXGAxIIO24FdDwO5kgkuY4HaCLVrkBTC6AGbYsGOB5CtI9rkIaKzaCN2X/FzMEVmKhmjeRmCg4AZjk9BmpH2fWgbhlvAZI1+lLfZjZsNKHZgCi9WwuCfIEUGTwcv3bhCtvIQ8TXCHMeGhQqGcd/w1rsd+8NqVuuWb6OD6Q9rKIdIfX3GAQgEMQrFgMEEnG3jWycrCP9FWwYfWrw99PnoKoHHzWPPwpLikZ/R1w2Dp/Q2M42z2Upxnz36etBnPs5hmMk0kdvNKqgKWjUYDbNpyxGWxg4GTuPMUzvbSa5N1dxQSPBE8naP3V0aFDuGVmDgqpGRirr7OopZOGRC3ZVlW/bLFC6p9UTl1BBxpIHUfaFT/AClbf4fspJoc3N1xLWM6TOe0mjJiTJJHd1YycL4mgyNuAXYt/pZtZvo+gS9oNBHZ41a9IcsBjfpt40a/5bvIou3ktZkhADF8oQFOMMQjlgN+uNvStNjXHDLaILpn/Q91pkJJ0BVtA6aOhLEr3s7aOhyae80ITw670oUb9GR5lOSrKFmJjA6BgNW+c/WDbYZDHLngN2kcczW06xSFVRiBglyAgI1ZXUSACwA3HnXXXBbqJZGlgkRIpBFIWZMJIyq4UgOSch1ORkd4Vt3OBH0eVv2WPDtJ8CfpS4x59R91R/tXMZsJyg730mASfzgJjP8Ao0fDFBiphNWHkJcXin+CT/bUMKsXI0IN1n/05PH0xQQHNj5upD6j8KCj80DF1J7x+AoKCxzJvTGQ0+un+FMX3oLcsn/kko9/+8VWeBcJvcQ3bRXz26RkJIlwxZYcdECz9oF7o7qjwG1T7gjg8nqf/wCxVp5JiPYcNfB0rZXIZsbAs9uQCegzpP8A8T5UGdXlrdfSYltY79RLCGtlW4kz9HwhOCJyI486dmKjJX3Ui1tfJ2wKX64kRJh9JcZknKomodvl9ZKrqAIPnttofC9xaL1ZuCsFHif8vsB4+G1SBlEfFJ5ZDiEW9hCwxkGeS5kEXxUsPdrBoMtHCL5RGqxXaiG4jSNe3RkjnQq6KqNOU/d8MDPUUMvBL2KaMPbXaTyvJ2brIitJI+qSQdrHKFBPebBIzjYbVeeB28kcJWUkuONMST+1mTUG+IIPxqXvYyslnqBGeKTMMjGQYLzBHofOgyW/sp4H7CRLqOQyRSiM3DEtLIwSOQaZivaFlA1kgjSNxS0vBL5ZzbGG7E04MrRm4z2wQ7u57fQ5Bx9o56VMz2QfjyoIJINV2khDs5MnZyzy9spYbRuYSQq5AxV9ggL31lcGSOXS3EYi0ZyMNJrjQn95FTSR5g0GUQcE4gJDZrBeK2h5exWfQnZsdDMAJxGQS+CAc77irFxq1eSKGO2W/a8DMdJupy0RhJhlbUZzGm7FQwO4YgdTVp4Tg3lsypiJuFTaIgSSuJbfK6+rZyoBwMafWoDk/XG7LHABGsF2ZrVzL2vZmbXoTUmp5AHwc4Daxv0NBSraz4kZpLSNbwyiORJImfOiKYqz7yOUUOQO8py29R6G5SWOEG7WeFjFFEJJQ8bMN0jUP3QRj7OxG+cVsllYtHfcTQyvM0ltbOgfBdVLXA7PYDIBxjO+GGc9TXLh1/8AFSnIx2oUn+L6ARj37igpl1wvicU0UDJfLLIjpEgmc6ozpMiqyylQuylgSBsM+FJXtlxMs9m4vSY4TM8DTsV7BCBqCmUo6jYALnpjG1a7wKKaO9t1nlTS78RMMJi0uo7YsSXLHWCDnZRsR1oljOsV/b6xhouDOzqR3gBLBsR1H2WGD5Ggzrk/gF2ssiNb3XfhilKw3ATVHIXCs2idAQdDDBydugrp+XuIdovYwXaJEZ2gUXIHZKjdnJ2Y+kdzBbSQOufEVpfFeyhmv8oXhj4TDlAxGqNGve6HG4JUY1DpnNKcuMv0WF1GmJLe+X7RYR4nQaDIepAUjJ3OknzoMge1v/owuyt99G7NgJDNLgQvp1d3tdfZNpUnu4OAegzS3ELDiXYsJkvGhSJJWV7kugiYkKxTtyCvcO2DgDcVe4rWabgIWRuyZeHakliY6HhWHaKbUuA2NjjP7RBHSrNbGPU4lGY2sLNHH8LvOhz6YNBjC8Dv3lNn2F0zwBX7HtspCCMoyntezQ/u4OdjgbU0uJZtUkUz3IfXmWOWWU5kVVAZ1ZiGbSFw2+wXBxithli13nGI11aiLD7ABcDSd1BBBI3O4I86zvn+3KcSuA8jStmLLMEBP1MfXQqr93hQQQAx0qw8gnFyf+m/5VXGxjap7kp8XB/6bflQQfM5zdS+/wDIUFNuPv8A4iT3/kK6gtFwcmmjNS81ypNNHlUeNBcLr/6K3vH++qlwFnd0haacwkEdl28wTH7ugPp0+mMVO3vFojwrsQ3fJG3ufNV7lqVVnQscAZ3oHPF4AtyQrygxd2I9tLmJcAaY2L5QeGFI8qb9lsw7SY6mDv8AXS951wQ7d/dhgYY7jA8qU41do1xKVOVLbHzpibkedA8eV8/rrgksH/zE/wBsAAP9v7QAA1ddql7LhbysjyzXBZd0YzzFkzsdDF8qSNsjwNRVtOg32JqycKu8gFj12wPEf3tQS0PBI2wxMxfu/WG4nMg0iRRiTtNSj62TYEDvmpGy5bt1AC9uuCWGm5uAAzZ1MAJMBjk5PU5OetDY3LeRUHwx+JqTacqM4+dAhb8r2ylSO2DKpRSLm4BVCQSgIl2XKg4G2w8qGXlq3OnuvqRmYP2svaAvjURLr174GRq8B5Ur2+obZp1FnzoIR+VLT7QV0kyT2yyyrKc4BDShtbDYdT4Cm/DeR7QI2pdSly4JZtWrOS2vOrVnfVnPrVl7PPX7q5o80FP41wKDPaLG7uukLK0srOMdAkrOXUDfofOqpdssRZzDcFiCHkDzh2VtOVaTXqePujutkVp13b7bqGHkf6VC8XsXlXMYRgOqttn49QfdQVvgNrb3YkZZLjOgRSRPPPjs+9hP1m8febuHKjJ86qnFkdJZ4i0io0jMyCRwj5JOWQHS2fUb1eOFWLxM5aPQSRtkH4BhjI9+9VXj10s+s9XR2UN46cnZvPHhQQZc9mYtUgiJyYu0cR5Jz+rDaeu/SjyXkhBBnuCCoQgzy4KLnCnv7qMnbpvTXBzinUEK+JoCi4k1mXtZ+1IwZO2k1lcAaS+rUVwBtnG1dMXzqdnYnHed2YnAAG7EnAAAA8AKk4Fix/e9Kcx6CAUx1H4UEKGqw8kpqnbp+qb8qrINWnkI/wCIf/pt8d1oKvxwYuJB6/lQ0XmD/My/zfkK6glnxmm8lS0vDzSVtwaSSRY0GWY4AoIeiOak+YuEyWkzQS41AK23Qhhkf36VEqhZgoGSaDi1Cppa74bJGxRhgjH370kICKCW4BwtriUIDpH7THoP+a1nh3DLaEKsaa3UY1Pn7h0zWbcELRxAKcMxz+Qz51oXLloypuWJO5znPx9aCw2wGMaRkny6HGahuIzlpMfnUp2rbLtt41E8QGHPX+/Wgd239+lP4hnxqPtnHjvUjG23iPeRQKe6gagyD0OaKJPA5oAmXI91RE2xz4jrjxFTRj2qGvVw39aCL4hwxc9sGk6HUR8+np+dZlzPY9me3gk7SKTO42KsdyGFbHw58b+BqE5s5SR0eS3xuCZYh59e0jHgQeo8aDE3lJ3zQG4bzobhCrFWGCDgjyIpHNAsLhvOjC8fpk03zXGgdLct1p/wXj727l1AyVK/gfy++ocGhFA7u7kyOznqTmupuhrqC7Smpj2ejPEYM+b/AOw1BSk1LchTab+E+rf7TQd7ZCP0kf8AoxZ+b1UuCSYnjI/eqd9rFxq4nN6LGPkv/NVjh0mJF99BOcx3Ba4c6ifs/gKimlI8TRuLSZmc+78BTMvQXnlMLs8hPU4Hn5/CtFs7xD+10HQeHv8AWso4fIVJ8AqLj44qxcN4llRGh7xJJ8fz/vFBdRcrrwp2/vxptxCQF+vwqOjugF7u+Nj1G46/lSqS7Z333J3oGfF+LNFHpiJ7Q9MDOPcPOq0kHFZSCdZxvgsoPvI/4p7x3iax57Fw07bKPFc+hGF952xVTuI5dRaaQxeOpV3P/uZyTQXvg/HponEU6MG2wCNz7vOrXfX+iIsepHlWfcl/SJJVklZniRCqMwwTkgjc7npWgceiHY91Q2R0z122FBQ+Kc+TxSDSuVG2Dnf/AIqS4LzObkYZFVsdQ3yyMVTX4zciQrpQBdmChGOBtsWBHzFTXCOJRXEvZ6NEgGoMV0k4IBBA7rdeqn4UGjcLTIPoBXTTKGIZCT1DI2k9d9/yoljJgMfEYGPSm14ofG58x+H9aDMPaLaxrcGRd+03bA/a6b+GdvDyqE4Pwg3B7qnHnipn2hWzRzAH7GgBc+YJz8anfZ8umPIFBV+LcrtAuoqflUEOzP7JxW1cbVXi0tjBB6/361jHFLcRysoORmgHRH5H5f8ANXbkTkyC9trhzqDI+lTnp3A2fmaoQrTvZNxcQ210GIHfDf8A6AflQZZNHpZl8mYfIkflXUNzJqkdvNmPzJNDQWqU045al03UePM/gaYz9etL8u/5lPeaCP53nLXsxPXI/wBoqIsm76++nnNb5u5v5vyFR9gfrF99A94hJ9Y3v/Km+aNfH6xvfTVqC88IjR4dWRq0qN/Mbb/jQWVuJZY1BIGsHOcHCnJO3QflUFwy9IhdB4DP34P5VO8DjJVZN98qfTrlj5Y/rQTn6SR3KKwYxtpYr0OcHP5VKRn349d/uqG4HwdreKfWBmR1dXGN1X/nPzqUsj3vHBHyx+eaCxWHComGWQHI3GB9/mffRzy5b5LGFCfA4GwI646Cm9nxArvn3/0qrc18zXEx+jWqsS2zMo+yCd8GgtCXUOoxI4cqcdxe6uPDIGM+lS80WUx/e1Ue24gLCGOB4GAQZ7Ubgs3UsOuSfHenlrzuurvYPu3FA7v+VILlhIMxyjbWvXI/eBGD7iKTPJoGHaQ6kOVcbY89lx1+f4UwveYjF2d5GG7Jn7OYFTjST3XHuO2fWre3FkeMMGBBx09aCGhuShZSQGPdAHjsx2z7qaScTTuMDjUcFSCOh3/7VDc7XDIElU4zIMeeApB/EU2tpRMMY7+QRt1BIGcee+/uNBN+1LgokERXHXBx4hgSB787io/hHCnto8L8vCpjnO7Jt5iP2FiIP8SMAcVQV5zmAxgnbfPpQaJYqJl0ybb+NZz7Q+ALbSBlOzeHkaL/AONph0FQ/H+OSXJBfwoIxTVl5alIhnxkefyqspVh4CfqZx/fQ0FbHU11cep99BQWqRRvnrTvlsf4hNvOo+V9z+FSXJ5zdIPQ0EBzT/m5v5vyFM+Fr9ao9ac81N/i5/5zTfgv65aDr9frH/mpvppa/b6x/wCY/jSOqgc2c2nPqCPn4/MVpXBdK8PjZdJbJ1bjbLY39w3rKyae2HEnjyBhlbqrdDjx9CPOg2RkQWww6kA6V/iJPhTGzmAJ23HlWZWHGH7eHLHQHXu5OAfs5q/QzjXjpnIPWgLzFxPRbMyk6hkD3n+lE5D4lb9Cy9og3zvlcbkE9STj5VFcVgL64/LvD1zVTg4M7xmRTjDFR7x038KDdZLmBo1+zpOME+Oc5x8NqiY7C17zRLHq2AJ3wc/84rFXjmUYJcY8CSMHzxmlIp54RqDumr1Pex+NBvjGKSMxSAMviMeFQHCYGiaWEnKpIQh/gJyM+oBx8KqPLnErqGF7mXLIx/aPiR9rHrjpVq4RcMYe0mOGIaV/IAgtigNz0IjAkRGJCnar7gTnHvyv3VB8l3jsXLFezRd30jI32UN57dBUU3Ncs6LKwXMaacEfn18qY3vOM7poUJGo/dH3+WaC2c2cXCWrxt+skVQR4qSSzZ+QHvBrONeaNcXTSEs7Ek4yT1pJkoDjFFY0CrQnFAUH0qZ4Icxze4fgah2NS/AfsS+4fgaCvnqffXVx6muoLBLnNSHJkmm7U+Qb8KiZn60twSbTMG9DQMeZ31Xc5Hi5pLgh+tHxpDiMmqV282P40twY/WigC6Hffx7x/GkdNHuZO8x/iP401c0ChYUTXRDQrQASRuOo3+Iq/wAkrCRd9mVXU+YdQfxqgk1oPKai8shGN57UnQPFo2JYD4EsPhQGmk0SQyNsC2h/c+2+/ng1MWVgqR6CuQSxb1ySf6fKoO5IlRl8fvBG1Wblu8WSNdXUgZ9D0P3igieIiNdIbQ8YyAsg3X0DdaK1jbSgaIVL4wGZiQo9Fz1q53XLVrMBl2B9CD+OaG25UhiBKyEn1A/KgiOLcJV7Psx0BQk9M4YZ+GKrvN3FOytCg2efuKPKMYyflgf6qvvFtIi7Iftdd+i+NYpzleGW7kG4WP6pQfAJsfmc0CXDn+pcehpjml+HMNLjPgaQJoDoaUz/AEpFWoxegNQOKBZKM8lAixqZ4G40S+4fgahmYGpDhbYWT3D86CLzua6uHU11BMSfdQW2zfCkzcKPOkJLvB7uw9aBq65J99ABjoaOTRBQFopo9ARQABQZoy0BSgCpjlHjH0W6SQnuHuvj90+PwOD8Kh6Cg2fmngJkzcW4HaYyyjpIPMfxY+dUW04gRkAlWydj4HxBFaL7ML8XViFb9ZAeyJ81G6H5HH+mor2m8sQRxNclxG/Rf/Ub93HicePhjegq8fOE8Zwy58/Ij0qU4ZzVPI+EU4x+0dh7v6Vnwu2HjmrdyLay3swg1hY+p3AO3XSOpoL/AMBja5mCDLBSGkbwwP2R6npj31lHO8RXiF4Dt9fIfgzah9xBr0hwrh8cEYjiUKo+8+ZPifWsA9qjg8VuceHZg+/s1oKmaEOaA0FAcSUtEjN0FN6NBcMhypwfx99BKQ8KckbHc024lbGNsEU7teYWGNQHhv8A30pLit2sp1DH3/34UEYKf2b91/cPzpkq+tOYWAVtxuBQNRXUNBQHkpMGjyGioKAVagauZcUDNt6ig4UoFzSaMKWWgQ6daPijSDNJxjr5UHMKAUbFcaC6+yPjQt7wo5wkyFT/ADL3l+PUfGmfN95cXXEGa7R0VdkiORpiztjz1dSR1+AxX+HXZiljlHVHVvkc1uHPvCEvrWG4iOJgA0beYIzpOPA0GYxcLjYjC433yPD0+6luN8NEFuZFyrKV0ODgqwIBIYb569POnPC+PJHG0jjePusmMMGGBp9Mnaqlxzj011JrlbbJ0oPsoPJR+Z3NBrfsl5+e4Y2d24aXGYZDgGQD7SN4FwNwepGfLJp/tm4eY+I9p4TRo3xXKH8B86pFrcvE6SxnDowdD5MpyK0D2z3qzvYXC/Zlti6/6irY+Gqgz1q7NdQUAk0ShYmi0HEUGaE1zCgHNcDRaFaA4oaAGuoDSda5WrpOtdHQGD1xpORcGhWgL40orUm9ctAs9dsBXCgbY+n5+dABoMUZ67P9/wBKAuK13krjOvhaAnvW8wQ/ytuv3HHwrJDU7ypxTsWliY9yZMf+4h1IfTxHxoJv2m8EVG+kx7LJjWPAsOhpjw3gvD7m3HZSyx3KlO1EhBypOlmVehUEjcHbx61K808SEvCz5iSNc+uQT9wqgcPlCyxsegZcn0zv91A/4twV4HKP1GcEdCPMehp1xu/7Wy4epPeh+kxH0XXGyfccfCtK47wxJbYFx3o8Lq8SMbZ8qyHicQWQgdKBvigpQHaitQFIoNNGCmgNAQigozUBoArgaEUFAYV1cprqBSQ70VTQydaCgVk3ApIHFKI1A4oCPvRQaEiiCgVBpbG1Nw1LLQF6VzClCfCiL5H4UACuKmjDxoQ3h8qCb4TZz3sbwRkd3TLg7AsMqBn1yar11bOjFJEKMOqsCD8j4etah7FrYN9LyMnEQz/8zVh5i5biuQUlVjv3XH219QSOnp0oMx4XzvcxoYpGE0TLpIYd4DzVxucfxZqH4q+ZM1I818ry2LgnvxMe5JgYJxnSy+DY8PH8ISZ84PoPu2oBOa40IxijnFAXwpNqU00Vx50CWK41xoM0HZrqChFAZa6uSuoDt1NAKGSi5oBU0oDmiGhFAVqTpXNEYUHUdTRBRgaBctk0DLmkwaOGoBTy8aOV8PA/nRWOfy9MUKny+IoND9jvGoYpJoJXCPMYzHq2DFQwK6ugO4wNs1qVxDgdP+PHrXmKXrWgcpe06SJRDeBpoxgLIN5EHTf99R8/fQXvnPh4mtHix1G3j3h3lPp0rAmHn4eFeh+IXaTWwkhcSK4yrKdiPH4jfY7isN5msuynfH2WJdfiTkfPNBHx9KOOlJxg+FKgfGgDP9+6kX60s2Kbk0AGgoTXYoAxXChNGjAzvQAlDVz4HyV9IXUjV1BTpBvRaO4yTRTQcKGgrhQdRWo+KBxigKoowFFWhoDA0ZaIKMBQKKKK+2/zocUIoEZsZ26USjEYNBQTvKfMj2jlWJMD/rE8B/GB+8PvGfSpPny3ASFwQclhkeIIDZz5f1qnEVJ33GXlt4IG/wDs6sN5g4C/IbfKgZx9N6MCKCEjG/8AeKOVGPf+NAk7d3FJCjyUXFAUUNSvLHBzdXKQjOGO/oK2e69lFr2OFXDY65oMCIp1wqwaaRY06k1J8Z5clhmMYBO+B61fORPZ9MjLM5IOxAHzoND5I5VW2gUHc4oatMC4UDyFdQePZOtBQyda4CgCjKKHFCKApNJsamuT+GpdX1vbyatErlW0nBxoZtjjbcCrHw3kWObjUtjqkW2jZ8nI1lURDpDY6kuN8dM0FCWhNa03s5spJrCSIzxwXEk8MkZfLZiinZWVyCRvCcjfqOm+TcN9mtk6qWM2TfXFts4/VxSTqv7P2sRrv76DJAaMGrV+BckcKmubu1IvBJas5ZtaaSmruhT1LYxnIFQfKPKlpfX2hROls0csyh2USBEEKjUVyBlpCfdigopehDVrFr7NLM8QngYzmBbaG4j0uNR1s6nfTvuhwKR5a5W4PePMixcShaBBI6zlEOls4wNznbxxQZWwzRB61pvE+WOGLwteIRR379qJBEBpYoQJdLyqNgg7PcgnrUxxL2V2X0d2R7lJVtu3EjMpj1aSdLZXzG4GNvGgxkjFARV+9nvKtpdWNzd3K3LmFwAkBGpl0I2FXG7ZY+NQ0fI9/Nqkt7OdoGd+yLmMNoDsFDAuCGAGD6g0EDAaUatK4P7M0bhU08izJeotzhA641xF9K6QCD9keNSnF/ZRai5s4Y5Jgspl7YlwSVjQNhe7sSxxnwBNBjLGuBrT+K8i8O/wlwJpLW2kuZrabtZBgdj9IGoO32SzW5Xcn7Y6HrC8+8rW8FvZ3dqJEW5z9S76zjTrV1bGcEdf5loGHs74ksF7G7dDlc++vSkN8jICCDXkUEg+INX7lDnaRB2bknHQ+lBc+dZ0S5jkOMBhvWicFuleJWHpWBc88dE2ApJ8f79aluQeezGoikPoDnwoN4FdVd4fzNE4+0K6g8vPXAUZ6MooCEURmpZ1xSLCgnfZ9dJFxO0kldY0WQlndgqqOzcbsTgdRV64RzLbRcdmnaVOwd5UEwYFO+keCXG2nK6c9ATWW20YJrYfZxwKHSrMgcnzH5UEpfcSjsm4UkzZRZ5pWnVXMIWeK5VPrtOnftQTvsASdsGlF45aWxgjku7YmS/uLjKzIVSGVriVXds4Ud9V36lsDNXKw5ctISpit40KghdKgYBBBwPDIJHxqRjt0X7KqvuAH4UGTcu8Yto+I8QmeeFIpllEUrSIEmw4/VsThselR/IfFbe3Msly6Q6LGMIruiPIJNbMIwxGo/Up08SK23FEeMEYIBoKDY80cPzHcJeW6FrEJpeeLWhXS6I41bONbZHmDVF9mPNIaa+nv7qMSS20ahpWjj1EBhpUd0EgY6b1d/aNwiIwsQgGPIVgcyYJHqaDU+HcSR+X7e2h4naWswWUTJJLHqaM9sDHpOSC2pfAGrXe8zWEsTwveWrRNZaShmiwZMEY+19vpt16V5+xXYoNM9kvFkj4beRfTbe0uHkBjaaRFweyjGrSx3GxHSoeHn42+qFrW1umSSUNca2xM3aOTINO2GJyMbYNUsiixR5NBuvJHOlp9Ft2kntrdmup2eEzIOzVu3xkM2QudJyfMU4k5zsu34bIbqAg/SBIRKh7LtEypkwe6CQFyfEisV4RDpOoHy/Gp+dtK6h1YAn/AFUEzzBy3wvtLaOSe3W5nu5pLmYTkoLYtPIFYltEbMpjUbA5zjO9W5r62ueJjMtrNCkBS3WKZH7MKUMjOi/ZLkoB6ReprOOC2T3S3AaVk7NCw0jqQCdxmoOw41PbNqRgfRskdQegI8se4mg1jmvla1m+yi6j0IG/kDkdP+1OuUvZlBCC8neYjxPQegqu+zTiUl45jl0dNJYKSTnTkksx37o3H471q45fXOdR66ug8iMfy75x5/KgzrmzkRdzEu/l4Vnz8n3CnIQnx2+dehjy8m3fbbHl4Y/pv55PwIeW0/fboR4b5Ock9c+X/bAYdwPhdyZdHfUYOevgK6tuHA0EmPBlOdh4HPht4+XifOhoP//Z',width=400,height=400)

Image makeameme.org

Much of Fearn's work was undertaken in his own time; to the despair of his wife, he was unwilling to throw anything away, and his study became so full of papers that it became difficult to open the door. Therefore Kagglers, don't be desperate if you're not receiving feedbacks, views or upvotes. Our work is in the Cloud. Maybe someday, anybody from other segments (even out of DS) take the time to get to know your work.

https://www.kaggle.com/mpwolke/co2-e-by-transport-sector

Kaggle Notebook Runner: Marília Prata @mpwolke
